In [298]:
import numpy as np 
import pandas as pd
#import polars as pl
import sys
from astropy.io import fits
import sqlite3
import plotly.graph_objects as go
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014
from astropy.coordinates import SkyCoord
import astropy.units as u
import astropy.units
sys.path.append('../../dash_gaia_orbits/')

import matplotlib.pyplot as plt
%matplotlib widget

# Package imports
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

In [299]:
conn = sqlite3.connect('/Users/cam/Desktop/astro_research/radcliffe/cluster_database/star_clusters.db')

# query = """
# SELECT * FROM star_clusters
# WHERE source = 'Hunt+2023'
# AND age_myr < 150
# AND U not NULL AND V not NULL AND W not NULL
# AND U_err not NULL AND V_err not NULL AND W_err not NULL
# AND U_err < 3 AND V_err < 3 AND W_err < 3
# AND U_err != 0 AND V_err != 0 AND W_err != 0
# AND n_stars >= 300
# """

query = """
SELECT * FROM star_clusters
WHERE source = 'Hunt+2023'
AND U not NULL AND V not NULL AND W not NULL
AND U_err not NULL AND V_err not NULL AND W_err not NULL
AND U_err < 5 AND V_err < 5 AND W_err < 5
AND U_err != 0 AND V_err != 0 AND W_err != 0
AND n_stars_rv >= 3
AND age_myr <= 150 or name = "Melotte_25" or name = "Melotte_22"
"""

pleiades_hyades = """ SELECT * FROM star_clusters WHERE name = 'Melotte_22' or name = 'Melotte_25' or name = 'NGC_2516' or name = 'NGC 2632'"""


df_q = pd.read_sql_query(query, conn)
df_ph = pd.read_sql_query(pleiades_hyades, conn)

In [300]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

m6 = df_hunt_gagne[df_hunt_gagne['family'] == 'm6']
cr135 = df_hunt_gagne[df_hunt_gagne['family'] == 'cr135']
ap = df_hunt_gagne[df_hunt_gagne['family'] == 'alphaPer']
gamma_vel = df_hunt_gagne[df_hunt_gagne['family'] == 'gamma_velorum']

df_hunt_partIII = pd.read_csv('/Users/cam/Downloads/hunt_part3_clusters.csv')
df_hunt_partIII_to_merge = df_hunt_partIII[['name', 'kind']]

ap = ap.merge(df_hunt_partIII_to_merge, on='name', how='left')
m6 = m6.merge(df_hunt_partIII_to_merge, on='name', how='left')
cr135 = cr135.merge(df_hunt_partIII_to_merge, on='name', how='left')

ap = ap.loc[ap['kind'] == 'o']
m6 = m6.loc[m6['kind'] == 'o']
cr135 = cr135.loc[cr135['kind'] == 'o']

sun = pd.DataFrame({'name' : ['Sun'], 'family' : ['Sun'], 'age_myr' : [8000], 'U' : [0.], 'V' : [0.], 'W' : [0.], 'x' : [0.], 'y' : [0.], 'z' : [0.], 'n_stars' : [1]})

In [301]:
df = pd.merge(left = df_q, right = df_hunt_partIII_to_merge, on='name', how='left', suffixes=('', '_y'))
df = df.loc[df['kind'] == 'o']

In [302]:
# Create a StarClusterData object
clusters = StarClusterData(df, data_name = 'Hunt+2023', min_size = 0, max_size = 6, color = 'black', opacity = 1., marker_style = 'circle')

# pleiades and hyades
ph = StarClusterData(df_ph, data_name = 'Pleiades, Hyades, NGC 2516, NGC 2632', min_size = 0, max_size = 12, color = 'black', opacity = 1., marker_style = 'circle')


# families
ap_data = StarClusterData(ap, data_name = '⍺Per', min_size = 0, max_size = 5, color = 'violet', opacity = 1., marker_style = 'circle')
m6_data = StarClusterData(m6, data_name = 'M6', min_size = 0, max_size = 5, color = 'cyan', opacity = 1., marker_style = 'circle')
cr135_data = StarClusterData(cr135, data_name = 'Cr135', min_size = 0, max_size = 5, color = 'orange', opacity = 1., marker_style = 'circle')
#gamma_vel_data = StarClusterData(gamma_vel, data_name = '𝝲Vel', min_size = 0, max_size = 7, color = 'lime', opacity = 1., marker_style = 'circle')

sc = StarClusterCollection([clusters, ap_data, m6_data, cr135_data, ph])

In [303]:
time_int = np.round(np.arange(0., -100.5, -.5), 1)

In [304]:
xyz_widths = (2000, 2000, 500)
plot_3d = StarClusters3DPlotter(
    data_collection = sc, 
    xyz_widths = xyz_widths, 
    figure_theme = 'gray', 
    trace_grouping_dict=None)


# save_name = '/Users/cam/Downloads/centered_aper.html'
#save_name = '/Users/cam/Downloads/gamma_vel_movie.html'

savefile = '/Users/cam/Downloads/large_cluster_traceback.html'
fig3d_m6 = plot_3d.generate_3d_plot(
    time = time_int,
    show = False, 
    save_name = savefile, 
    static_traces = None, 
    static_traces_times = None, 
    static_traces_legendonly=False,
    focus_group = None,
    fade_in_time = 20, # Myr,
    fade_in_and_out = True
)

In [305]:
clusters.df_int.to_csv('/Users/cam/Downloads/large_cluster_sample_int.csv', index=False)

In [306]:
clusters.df

,source,name,age_myr,l,b,d,x,y,z,U,...,W,x_err,y_err,z_err,U_err,V_err,W_err,n_stars,n_stars_rv,kind
2,Hunt+2023,ASCC_19,7.449458,204.842134,-19.394344,352.079120,-301.527630,-139.254993,-116.828260,-18.650816,...,-6.745320,0.429771,0.292802,0.309928,1.742597,0.800040,0.696690,61.0,23.0,o
3,Hunt+2023,ASCC_20,12.542164,201.538906,-17.604146,361.207595,-320.367905,-126.779356,-108.454265,-27.147193,...,-9.300209,0.885527,0.327817,0.267973,0.553852,0.228329,0.189088,194.0,52.0,o
4,Hunt+2023,ASCC_21,6.898220,200.066258,-16.523933,340.734410,-307.180237,-111.093713,-96.945485,-17.678163,...,-3.712033,0.296035,0.360332,0.166014,0.862013,0.300683,0.285509,116.0,32.0,o
6,Hunt+2023,ASCC_36,127.410872,234.209535,-4.570470,952.988720,-554.326953,-771.352334,-76.972117,-19.865801,...,-6.991145,5.455604,6.854048,0.656403,2.062342,2.822403,0.271358,29.0,3.0,o
7,Hunt+2023,ASCC_51,125.805876,286.520243,-13.972474,544.834742,150.943380,-506.725617,-131.491981,-25.198718,...,-2.823185,0.788039,1.198474,0.422074,0.261402,0.770934,0.214990,172.0,37.0,o
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,Hunt+2023,UPK_640,9.619756,343.331615,4.637148,173.862580,165.952955,-49.663222,14.878768,-4.325893,...,-4.221809,0.240055,0.298066,0.166911,0.478211,0.155781,0.052641,764.0,119.0,o
876,Hunt+2023,XDOCC_9,144.116853,59.598336,2.289598,549.272794,276.804843,473.910042,22.112301,7.212256,...,-11.196811,1.325075,1.893753,0.614223,0.654714,0.972544,0.074031,46.0,16.0,o
877,Hunt+2023,ZHBJZ_1,13.495854,218.464789,-2.158869,283.655598,-221.643659,-176.821748,-8.285895,-17.938806,...,-10.979492,0.609558,0.475140,0.216740,0.913801,0.717308,0.070737,490.0,43.0,o
880,Hunt+2023,vdBergh_83,6.283996,236.444661,-14.323189,903.690775,-484.103588,-729.458194,-224.034543,-32.453435,...,-14.265089,5.198281,7.874486,2.328685,2.783865,3.979086,1.400236,92.0,5.0,o
